## 열을 행 방향으로 이동

### Shift 

In [304]:
import pandas as pd

In [305]:
data = [
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가': 100, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  99, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가': 100, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  99, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75} 
]

indexes = ['20200615', '20200616', '20200717', '20200718',
            '20200815', '20200816', '20200917', '20200918']


df_first = pd.DataFrame(data, index=indexes)
df_first.index = pd.to_datetime(df_first.index)
df_first.index

DatetimeIndex(['2020-06-15', '2020-06-16', '2020-07-17', '2020-07-18',
               '2020-08-15', '2020-08-16', '2020-09-17', '2020-09-18'],
              dtype='datetime64[ns]', freq=None)

#### 전날 종가와 차이

In [306]:
df_first['전일종가'] = df_first['종가'].shift(periods=1)
df_first['3일전 종가'] = df_first['종가'].shift(periods=3)
df_first

,시가,고가,저가,종가,전일종가,3일전 종가
2020-06-15,100,110,90,105,NaN,NaN
2020-06-16,100,112,80,95,105.0,NaN
2020-07-17,99,115,70,85,95.0,NaN
2020-07-18,70,80,60,75,85.0,105.0
2020-08-15,100,110,90,105,75.0,95.0
2020-08-16,100,112,80,95,105.0,85.0
2020-09-17,99,115,70,85,95.0,75.0
2020-09-18,70,80,60,75,85.0,105.0


In [307]:
conditions = df_first['종가'] > df_first['전일종가']
conditions

2020-06-15    False
2020-06-16    False
2020-07-17    False
2020-07-18    False
2020-08-15     True
2020-08-16    False
2020-09-17    False
2020-09-18    False
dtype: bool

In [308]:
# df_first[conditions]
# df_first[conditions]['시가'] # 종가 올라간 날짜의 시가의 시리즈
# df_first[conditions][['시가']] 종가 올라간 날짜의 시가의 데이터 프레임

In [309]:
df_first['종가'] - df_first['전일종가'] >= 10

2020-06-15    False
2020-06-16    False
2020-07-17    False
2020-07-18    False
2020-08-15     True
2020-08-16    False
2020-09-17    False
2020-09-18    False
dtype: bool

### 수익률 계산 with shift

In [310]:
df_first['종가'] / df_first['시가']

2020-06-15    1.050000
2020-06-16    0.950000
2020-07-17    0.858586
2020-07-18    1.071429
2020-08-15    1.050000
2020-08-16    0.950000
2020-09-17    0.858586
2020-09-18    1.071429
dtype: float64

## rolling()
 - Shift와 연산 가능

In [311]:
# 샘플 데이터 생성 
data = { 'value': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] }


df_second = pd.DataFrame(data)
df_second

,value
0,10
1,20
2,30
3,40
4,50
5,60
6,70
7,80
8,90
9,100


In [312]:
# df_second.rolling(window=3)
df_second['value'].rolling(window=3).mean() # 해당하는 값을 구할때 대표값을 자동으로 구한다 > shift와의 차이.

0     NaN
1     NaN
2    20.0
3    30.0
4    40.0
5    50.0
6    60.0
7    70.0
8    80.0
9    90.0
Name: value, dtype: float64

In [313]:
df_second['rolling3'] = df_second['value'].rolling(window=3).mean()
df_second['rolling6'] = df_second['value'].rolling(window=6).mean()

In [314]:
df_second

,value,rolling3,rolling6
0,10,NaN,NaN
1,20,NaN,NaN
2,30,20.0,NaN
3,40,30.0,NaN
4,50,40.0,NaN
5,60,50.0,35.0
6,70,60.0,45.0
7,80,70.0,55.0
8,90,80.0,65.0
9,100,90.0,75.0


### resampling()
- shift + like groupby (groupyby는 연산도 같이 된다)

In [315]:
df_resample_first = pd.read_excel('../datasets/ss.xlsx', index_col=0)
df_resample_first.head()

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2021/08/13,74400,-2600,-3.38,75800,76000,74100,61270643,4575267536355,444151821720000,5969782550
2021/08/12,77000,-1500,-1.91,77100,78200,76900,42365223,3276635421700,459673256350000,5969782550
2021/08/11,78500,-1700,-2.12,79600,79800,78500,30241137,2389977254924,468627930175000,5969782550
2021/08/10,80200,-1300,-1.60,82300,82400,80100,20362639,1643107615500,478776560510000,5969782550
2021/08/09,81500,0,0.00,81500,82300,80900,15522581,1267668377900,486537277825000,5969782550


In [316]:
df_resample_first.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 2021/08/13 to 2021/02/15
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      127 non-null    int64  
 1   대비      127 non-null    int64  
 2   등락률     127 non-null    float64
 3   시가      127 non-null    int64  
 4   고가      127 non-null    int64  
 5   저가      127 non-null    int64  
 6   거래량     127 non-null    int64  
 7   거래대금    127 non-null    int64  
 8   시가총액    127 non-null    int64  
 9   상장주식수   127 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 10.9+ KB


In [317]:
df_resample_first.index

Index(['2021/08/13', '2021/08/12', '2021/08/11', '2021/08/10', '2021/08/09',
       '2021/08/06', '2021/08/05', '2021/08/04', '2021/08/03', '2021/08/02',
       ...
       '2021/02/26', '2021/02/25', '2021/02/24', '2021/02/23', '2021/02/22',
       '2021/02/19', '2021/02/18', '2021/02/17', '2021/02/16', '2021/02/15'],
      dtype='object', name='일자', length=127)

In [318]:
df_resample_first.index = pd.to_datetime(df_resample_first.index)

In [319]:
df_resample_first = df_resample_first.sort_index()

In [320]:
# df_resample_first.resample('MS').first()
df_resample_first.resample('ME').first()


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2021-02-28,84200,2600,3.19,83800,84500,83300,23529706,1978336504100,502655690710000,5969782550
2021-03-31,83600,1100,1.33,85100,85300,83000,33498180,2823604648915,499073821180000,5969782550
2021-04-30,82900,1500,1.84,82500,83000,82000,18676461,1542484638200,494894973395000,5969782550
2021-05-31,81700,200,0.25,81000,82400,81000,15710336,1284767138340,487731234335000,5969782550
2021-06-30,80600,100,0.12,80500,81300,80100,14058401,1135461759600,481164473530000,5969782550
2021-07-31,80100,-600,-0.74,80500,80600,80000,13382882,1073188213795,478179582255000,5969782550
2021-08-31,79300,800,1.02,79200,79500,78700,11739124,929153637896,473403756215000,5969782550


In [321]:
# how = {
#     '시가' : 'first',
#     '고가' : max,
#     '저가' : min,
#     '종가' : 'last',
#     '거래량' : sum
#     #'등락률' : mean
# }
import numpy as np
how = {
    '시가' : 'first',
    '고가' : np.max,
    '저가' : np.min,
    '종가' : 'last',
    '거래량' : sum,
    '등락률' : np.mean
}
df_resample_first.resample('ME').apply(how)

/tmp/ipykernel_97622/1240883739.py:18: FutureWarning: The provided callable <function max at 0x7fe1ec9765c0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_resample_first.resample('ME').apply(how)
/tmp/ipykernel_97622/1240883739.py:18: FutureWarning: The provided callable <function min at 0x7fe1ec976700> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df_resample_first.resample('ME').apply(how)
/tmp/ipykernel_97622/1240883739.py:18: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_resample_first.resample('ME').apply(how)
/tmp/ipykernel_97622/1240883739.py

,시가,고가,저가,종가,거래량,등락률
일자,,,,,,
2021-02-28,83800,86000,81000,82500,255020740,0.133000
2021-03-31,85100,85300,80600,81400,387612356,-0.055909
2021-04-30,82500,86200,81500,81500,372938171,0.009545
2021-05-31,81000,83500,78400,80500,352211074,-0.059474
2021-06-30,80500,83000,79600,80700,333099465,0.015455
2021-07-31,80500,81300,78100,78500,275886253,-0.121364
2021-08-31,79200,83300,74100,74400,263311167,-0.520000


### 일단위

In [322]:
df_resmaple_third = df_resample_first.resample('3D').apply(how)
df_resmaple_third.head(5)

/tmp/ipykernel_97622/494662059.py:1: FutureWarning: The provided callable <function max at 0x7fe1ec9765c0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_resmaple_third = df_resample_first.resample('3D').apply(how)
/tmp/ipykernel_97622/494662059.py:1: FutureWarning: The provided callable <function min at 0x7fe1ec976700> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df_resmaple_third = df_resample_first.resample('3D').apply(how)
/tmp/ipykernel_97622/494662059.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_resmaple_third = df_resample_first.resamp

,시가,고가,저가,종가,거래량,등락률
일자,,,,,,
2021-02-15,83800.0,86000.0,83000.0,83200.0,62320541,0.673333
2021-02-18,83200.0,83600.0,81000.0,82600.0,47208562,-0.355000
2021-02-21,83800.0,84200.0,81100.0,82000.0,46007200,-0.360000
2021-02-24,81800.0,85400.0,81300.0,82500.0,99484437,0.246667
2021-02-27,NaN,NaN,NaN,NaN,0,NaN


In [323]:
df_resample_first.resample('12H').apply(how).head(5)

/tmp/ipykernel_97622/1101196726.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resample_first.resample('12H').apply(how).head(5)
/tmp/ipykernel_97622/1101196726.py:1: FutureWarning: The provided callable <function max at 0x7fe1ec9765c0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_resample_first.resample('12H').apply(how).head(5)
/tmp/ipykernel_97622/1101196726.py:1: FutureWarning: The provided callable <function min at 0x7fe1ec976700> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df_resample_first.resample('12H').apply(how).head(5)
/tmp/ipykernel_97622/1101196726.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a f

,시가,고가,저가,종가,거래량,등락률
일자,,,,,,
2021-02-15 00:00:00,83800.0,84500.0,83300.0,84200.0,23529706,3.19
2021-02-15 12:00:00,NaN,NaN,NaN,NaN,0,NaN
2021-02-16 00:00:00,84500.0,86000.0,84200.0,84900.0,20483100,0.83
2021-02-16 12:00:00,NaN,NaN,NaN,NaN,0,NaN
2021-02-17 00:00:00,83900.0,84200.0,83000.0,83200.0,18307735,-2.00


### 수익율

In [324]:
data = {'삼성전자': [52200, 52300, 52900, 52000, 51700], 
           'LG전자': [68200, 67800, 68800, 67500, 66300]}

In [325]:
df_first = pd.DataFrame(data).pct_change(periods=2)

In [326]:
df_first.pct_change(periods=2)

,삼성전자,LG전자
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,-2.691601,-5.130329


In [327]:
df_first.cumprod()

,삼성전자,LG전자
0,NaN,NaN
1,NaN,NaN
2,0.013410,0.008798
3,-0.000077,-0.000039
4,0.000002,0.000001


In [328]:
data = {"DailyReturn": [0.01, 0.02, -0.005, 0.015]}

df_second = pd.DataFrame(data)
df_second

,DailyReturn
0,0.010
1,0.020
2,-0.005
3,0.015


In [ ]:
# 율을 할때는 1을 더해야함, 왜냐하면 소수 곱하기 소수는 더 작아지기 떄문에
 
(1 + df_second['DailyReturn']).cumprod()

0    1.010000
1    1.030200
2    1.025049
3    1.040425
Name: DailyReturn, dtype: float64